In [5]:
import chess
import chess.pgn
import chess.uci
import random

In [6]:
import pandas as pd
import numpy as np

In [7]:
STOCKFISH = '/usr/bin/stockfish'
DEPTH = 4

In [10]:
def make_score(engine, i):
    fen = positions.iloc[i].fen.strip()
    engine.ucinewgame()
    board = chess.Board(fen=fen)
    move = random.choice(list(board.legal_moves))
    board.push(move)
    fen = board.fen()
    orig_fen = fen
    engine.position(board)
    engine.go(depth=DEPTH)
    
    info_handler = engine.info_handlers[0]
    score = info_handler.info["score"][1].cp
    if score is None:
        if info_handler.info["score"][1].mate != 0:
            return None
        if info_handler.info["score"][1].mate > 0:
            score = 5000
        else:
            score = -5000

    wtm = board.turn
    if not wtm: score *= -1

    pv = info_handler.info["pv"]
    if not pv.get(1): return None
    for move in pv[1]:
        board.push(move)
    fen = board.fen()
    
    pv = info_handler.info['pv']
    if not pv.get(1): return None
    best_move = pv[1][0].uci()
    
    return orig_fen, fen, score, best_move

In [11]:
def make_engine():
    engine = chess.uci.popen_engine(STOCKFISH)
    info_handler = chess.uci.InfoHandler()
    engine.info_handlers.append(info_handler)
    return engine

In [14]:
from multiprocessing import Queue
from multiprocessing import Manager, Process
from time import time
from tqdm import tqdm
import os

#features_test_speed = bcolz.carray(np.empty((0,8,8,14)), rootdir='../slonik_data/features_test_speed.bc', chunklen=1024, mode='w')

n = 1
step = 25000

# for num_workers in range(10,16):
columns = ('orig_fen', 'fen', 'score', 'move')
try:
    data = pd.read_pickle('/ssd/slonik_data/sf_move_scores_4dep.pkl')
except FileNotFoundError:
    data = pd.DataFrame(columns=columns)

positions = pd.read_pickle('/ssd/slonik_data/positions.pkl')
   
now = time()

def save(temp_data, data):
    temp_df = pd.DataFrame(temp_data, columns=columns)
    data = pd.concat([data, temp_df], ignore_index=True)
    data.to_pickle('/ssd/slonik_data/sf_move_scores_4dep.pkl')
    return data

def do_work(proc_num, work, output):
    engine = make_engine()
    while True:
        i = work.get()
        if i is None: 
            return
        res = make_score(engine, i)
        output.put(res)

for s in tqdm(range(1, len(positions), step)):
    num_workers = 10
    manager = Manager()
    work = manager.Queue(num_workers)
    output = Queue()
    pool = []
    for i in range(num_workers):
        p = Process(target=do_work, args=(i, work, output))
        p.start()
        pool.append(p)

    end = min(s + step, len(positions))
    num_fen = end - s
    for i in range(s, end):
        work.put(i)

    results = []
    for i in range(num_fen):
        res = output.get()
        if res is not None:
            results.append(res)

    for i in range(num_workers): 
        work.put(None)

    for p in pool:
        p.join()
        
    data = save(results, data)

    print(num_workers, "workers", len(results), "results", (time() - now), 's')

  3%|▎         | 1/29 [00:49<23:10, 49.68s/it]

10 workers 24682 results 49.67884159088135 s


  7%|▋         | 2/29 [01:38<22:10, 49.27s/it]

10 workers 24628 results 98.54144859313965 s


 10%|█         | 3/29 [02:27<21:17, 49.13s/it]

...

10 workers 24644 results 1354.8160281181335 s


 97%|█████████▋| 28/29 [23:27<00:50, 50.25s/it]

10 workers 24589 results 1407.124933719635 s


100%|██████████| 29/29 [23:29<00:00, 48.61s/it]

10 workers 745 results 1409.8157935142517 s


In [15]:
# sf_scores12 = data
sf_move_scores4 = pd.read_pickle('/ssd/slonik_data/sf_move_scores_4dep.pkl')
s = np.array(sf_move_scores4.score,dtype='int32')
sf_move_scores4.score = 2 * ((s-s.min()) / (s.max()-s.min())) - 1

In [23]:
sf_move_scores4.to_pickle('/ssd/slonik_data/sf_scores_12dep.pkl')